In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from vastai.api import VastClient
from io import StringIO
import time
from IPython.display import clear_output

In [3]:
vast = VastClient().authenticate()

Initializing vast.ai client with api_key from /home/sam/.vast_api_key.


In [14]:
vast.get_running_instances()

[]

In [4]:
def monitor_file(log_file, popen=None, n_lines=5, update_freq_s=1):
    while True:
        try:
            clear_output(wait=True)
            print(remote['tail'](log_file,'-n%i'%n_lines))
            if popen is not None:
                ret = popen.proc.poll()
                if ret is not None:
                    print("Process has exited with code:",popen.proc.poll())
                    break
            time.sleep(update_freq_s)
        except KeyboardInterrupt:
            break
        except NameError:
            break

In [11]:
instance = vast.get_running_instances()[0]
remote = instance.pb_remote
instance

464626: running   running              $0.1131/hr ssh4.vast.ai:24626  656.4↑ 591.7↓  flops:13.698048T
	1X 10.9GB GTX 1080 Ti, 5.333328/16X 31.8GB Xeon® E5-2630 v3 
	Successfully loaded tensorflow/tensorflow:1.15.2-gpu-py3-jupyter

## Check mongo status

In [6]:
print(remote['tail']('/var/log/mongodb/mongodb.log','-n5'))

2020-01-30T07:10:06.918+0000 I INDEX    [thread3] build index done.  scanned 0 total records. 0 secs
2020-01-30T07:10:15.031+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:41432 #3 (3 connections now open)
2020-01-30T07:10:15.145+0000 I NETWORK  [conn3] received client metadata from 127.0.0.1:41432 conn: { driver: { name: "PyMongo", version: "3.9.0" }, os: { type: "Linux", name: "Linux", architecture: "x86_64", version: "3.8.11" }, platform: "CPython 3.6.8.final.0" }
2020-01-30T07:10:15.253+0000 I NETWORK  [listener] connection accepted from 127.0.0.1:41434 #4 (4 connections now open)
2020-01-30T07:10:15.452+0000 I NETWORK  [conn4] received client metadata from 127.0.0.1:41434 conn: { driver: { name: "PyMongo", version: "3.9.0" }, os: { type: "Linux", name: "Linux", architecture: "x86_64", version: "3.8.11" }, platform: "CPython 3.6.8.final.0" }



In [7]:
print(instance.ssh_connection_command)

ssh root@ssh4.vast.ai -p 24626 -i /home/sam/.ssh/vastai


## Install model and download data

In [14]:
install_script_target='/root/install_script.sh'
install_script = StringIO("""#!/bin/sh
INSTALL_LOG=/root/install.log
echo avian_vocalizations INSTALL BEGIN `date` >> $INSTALL_LOG 2>&1
cd ~
[ ! -d avian_vocalizations ] && git clone https://github.com/samhiatt/avian_vocalizations.git  >> $INSTALL_LOG 2>&1
cd avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
pip install pipenv  >> $INSTALL_LOG 2>&1
pipenv lock -r > requirements.txt
pip install -r requirements.txt >> $INSTALL_LOG 2>&1
pip install -e . >> $INSTALL_LOG 2>&1
download_data >> $INSTALL_LOG 2>&1
echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
""")
# ("""pipenv install >> $INSTALL_LOG 2>&1
# pipenv run python setup.py develop >> $INSTALL_LOG 2>&1
# pipenv run download_data >> $INSTALL_LOG 2>&1
# echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
# """)
# monitor_output(mongo_instance, install_script)
remote.sftp.putfo(install_script,install_script_target)
remote['chmod']('+x',install_script_target)
p=remote.session().popen(install_script_target)
p

In [ ]:
monitor_file('/root/install.log', p)

## Start hyperopt worker

In [12]:
hyperopt_worker_log="/root/hyperopt_worker.log"
start_hyperopt_worker_target='/root/avian_vocalizations/start_hyperopt_worker.sh'
start_hyperopt_worker="#!/bin/sh\ncd /root/avian_vocalizations\n"\
    "export LC_ALL=C.UTF-8\n"\
    "export LANG=C.UTF-8\n"\
    "hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations "\
    " --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=."\
    " >> %s 2>&1\n"%hyperopt_worker_log

# print(start_hyperopt_worker)
remote.sftp.putfo(StringIO(start_hyperopt_worker),start_hyperopt_worker_target)
remote['chmod']('+x',start_hyperopt_worker_target)
pworker=remote.session().popen(start_hyperopt_worker_target)

In [13]:
monitor_file(hyperopt_worker_log, pworker)

  File "/home/sam/Downloads/avian_vocalizations/avian_vocalizations/evaluator.py", line 83, in ModelEvaluator
  File "/usr/local/lib/python3.6/dist-packages/hyperopt/mongoexp.py", line 943, in __setitem__
    self.handle.set_attachment(
AttributeError: 'Attachments' object has no attribute 'handle'
INFO:hyperopt.mongoexp:exiting with N=9223372036854775806 after 1 consecutive exceptions

Process has exited with code: 0


In [ ]:
# vast.stop_all_instances()

In [62]:
list(remote.pgrep('hyperopt'))

[]